#  Karamoja Food Security – Data Cleaning & Analysis

This notebook covers the **data preparation and exploratory analysis** for crop yields, population, and area harvested in Karamoja, Uganda.  
Steps follow the rubric: data connection, cleaning, feature engineering, and statistical analysis.


In [1]:

import pandas as pd
import numpy as np



yields = pd.read_csv("yields.csv")         # contains District, Subcounty, Crop, Yield_KgHa, Area_Ha
population = pd.read_csv("population.csv") # contains District, Subcounty, Population


In [2]:
yields.head()



,District,Subcounty,Crop,Yield_KgHa,Area_Ha
0,Kotido,SubcountyA,Sorghum,1299.3,156
1,Kotido,SubcountyA,Maize,772.3,121
2,Kotido,SubcountyB,Sorghum,977.6,264
3,Kotido,SubcountyB,Maize,863.8,380
4,Kotido,SubcountyC,Sorghum,797.8,409


In [3]:
population.head()

,District,Subcounty,Population
0,Kotido,SubcountyA,11873
1,Kotido,SubcountyB,10675
2,Kotido,SubcountyC,5161
3,Kaabong,SubcountyA,9297
4,Kaabong,SubcountyB,5995


##  Step 2: Inspect Data

We check for:
- Missing values  
- Duplicates  
- Data type mismatches  
- Spelling inconsistencies in district/subcounty names


In [4]:
print(yields.info())
print()
print(yields.isnull().sum())
print("\n")
print(population.info())
print()
print(population.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   District    30 non-null     object 
 1   Subcounty   30 non-null     object 
 2   Crop        30 non-null     object 
 3   Yield_KgHa  30 non-null     float64
 4   Area_Ha     30 non-null     int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 1.3+ KB
None

District      0
Subcounty     0
Crop          0
Yield_KgHa    0
Area_Ha       0
dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   District    15 non-null     object
 1   Subcounty   15 non-null     object
 2   Population  15 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 492.0+ bytes
None

District      0
Subcounty     0
Population    0
dtype: int64


##  Step 3: Clean Data

1. Standardize names (remove spaces, unify case)  
2. Handle missing values by imputing with **district mean**  
3. Drop duplicates  
4. Ensure numeric fields are properly typed  


In [5]:

yields['District'] = yields['District'].str.strip().str.title()
yields['Subcounty'] = yields['Subcounty'].str.strip().str.title()
population['District'] = population['District'].str.strip().str.title()
population['Subcounty'] = population['Subcounty'].str.strip().str.title()

yields['Yield_KgHa'] = yields.groupby('District')['Yield_KgHa'].transform(
    lambda x: x.fillna(x.mean())
)

yields['Yield_KgHa'] = pd.to_numeric(yields['Yield_KgHa'], errors='coerce')
yields['Area_Ha'] = pd.to_numeric(yields['Area_Ha'], errors='coerce')
population['Population'] = pd.to_numeric(population['Population'], errors='coerce')

# Drop duplicates
yields = yields.drop_duplicates()
population = population.drop_duplicates()


## Step 4: Feature Engineering

We add useful metrics:
- **Total Production (Kg)** = Yield × Area  
- **Productivity per Person (Kg/person)** = Total Production ÷ Population  
- **Yield Ratio** = Sorghum ÷ Maize yields per subcounty  


In [6]:

data = yields.merge(population, on=["District", "Subcounty"], how="left")


data['Total_Production'] = data['Yield_KgHa'] * data['Area_Ha']
data['Prod_Per_Capita'] = data['Total_Production'] / data['Population']

pivot = data.pivot_table(index=['District','Subcounty'],
                         columns='Crop', values='Yield_KgHa')
pivot = pivot.reset_index()
pivot['Yield_Ratio_Sorghum_Maize'] = pivot['Sorghum'] / pivot['Maize']


##  Step 5: Exploratory Data Analysis

We compute descriptive statistics and compare yields between Sorghum and Maize.  


In [7]:

print(data.groupby("Crop")['Yield_KgHa'].describe())


top_districts = data.groupby("District")['Yield_KgHa'].mean().sort_values(ascending=False)
print("Top Districts by Average Yield:\n", top_districts.head())

print("Bottom Districts by Average Yield:\n", top_districts.tail())


         count         mean         std    min      25%     50%      75%  \
Crop                                                                       
Maize     15.0   678.380000  197.813477  372.1   518.20   695.0   815.85   
Sorghum   15.0  1133.146667  181.684860  794.7  1054.75  1103.5  1274.00   

            max  
Crop             
Maize    1023.9  
Sorghum  1364.5  
Top Districts by Average Yield:
 District
Nakapiripirit    960.500000
Amudat           933.066667
Kotido           902.033333
Kaabong          896.800000
Moroto           836.416667
Name: Yield_KgHa, dtype: float64
Bottom Districts by Average Yield:
 District
Nakapiripirit    960.500000
Amudat           933.066667
Kotido           902.033333
Kaabong          896.800000
Moroto           836.416667
Name: Yield_KgHa, dtype: float64


##  Step 6: Statistical Test

We test if Sorghum and Maize yields are **significantly different** using a paired t-test.  


In [8]:
import scipy.stats as stats


In [9]:

sorghum = pivot['Sorghum'].dropna()
maize = pivot['Maize'].dropna()

t_stat, p_val = stats.ttest_rel(sorghum, maize, nan_policy='omit')
print(f"Paired t-test Sorghum vs Maize: t={t_stat:.3f}, p={p_val:.3f}")

if p_val < 0.05:
    print("correct!! Significant difference between Sorghum and Maize yields.")
else:
    print("not correct! No significant difference between Sorghum and Maize yields.")


Paired t-test Sorghum vs Maize: t=5.497, p=0.000
correct!! Significant difference between Sorghum and Maize yields.


##  Step 7: Save Cleaned Data

Exporting Cleadned dataset for visualization
  


In [10]:
data.to_csv("clean_food_security.csv", index=False)


#  Findings & Recommendations

1. **Sorghum vs Maize**  
   - Sorghum generally performs better across most districts.  
   - The t-test shows whether this difference is statistically significant.  

2. **High-Risk Districts**  
   - Districts with **high population but low productivity per capita** (e.g., Kotido, Moroto) are at higher food security risk.  

3. **Opportunities**  
   - Districts with large land but low yield should be targeted for agricultural support.  
   - Areas with good sorghum yields could be prioritized for scaling sorghum production.  

